### Вычисление рейтингов игроков Dota 2 с помощью Glicko-2

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import glicko2 as g2
import math
import pprint

In [2]:
match_outcomes = pd.read_csv('/data/notebook_files/match_outcomes.csv')

match_outcomes.head()

,match_id,account_id_0,account_id_1,account_id_2,account_id_3,account_id_4,start_time,parser_version,win,rad
0,1636204962,34549,0,0,-51743434,-120875154,1437014585,12,1,0
1,1636204962,0,61598,138825,0,207232,1437014585,12,0,1
2,1636322679,0,-44943233,-240360907,19599,0,1437019968,12,0,0
3,1636322679,-97530201,0,0,0,-116349387,1437019968,12,1,1
4,1637385965,0,0,0,104738,0,1437052551,12,1,0


In [3]:
unique_acc_ids = pd.DataFrame(match_outcomes.iloc[:,1:6].unstack().unique(),
                              columns=['account_id'])
print('Number of unique account ids:', unique_acc_ids.shape[0],'\n')
print(unique_acc_ids.head())

Number of unique account ids: 834226 

   account_id
0       34549
1           0
2   -97530201
3  -123447796
4  -108454938


In [4]:
rating_dict = dict()
for i in unique_acc_ids.values.ravel():
    rating_dict[i] = g2.Player()

In [5]:
len(rating_dict.keys())

834226

In [6]:
# Each account_id is a key with the value being the Rating object
pprint.pprint(str(rating_dict)[:253]+'}')

('{34549: <glicko2.glicko2.Player object at 0x7fc124748700>, 0: '
 '<glicko2.glicko2.Player object at 0x7fc124748d00>, -97530201: '
 '<glicko2.glicko2.Player object at 0x7fc124748790>, -123447796: '
 '<glicko2.glicko2.Player object at 0x7fc124748c40>, -108454938: <gl}')


In [2]:
def replace_anonymous(team_ids, rating_dict, team_name):    
    team_dict = dict()
    non_anon = []
    
    sum_rating = 0
    sum_rd = 0
    
    non_anons = 0
    for i in team_ids:
        if i != 0 and i in rating_dict.keys():
            sum_rating += rating_dict[i].getRating()
            sum_rd += rating_dict[i].getRd()
            non_anons += 1
    
    if non_anons == 0:
        for e in range(5):
            team_dict[team_name + str(e)] = g2.Player()
        return team_dict
        
    mean_rating = sum_rating/non_anons
    mean_rd = sum_rd/non_anons

    for e,i in enumerate(team_ids):
        
        if i == 0 or i not in rating_dict.keys():
            team_dict[team_name + str(e)] = g2.Player(rating=mean_rating, rd=mean_rd)
        else:
            team_dict[i] = rating_dict[i]
        
    return team_dict

def update_ratings(new_ratings, rating_dict):

    for key in new_ratings.keys():
        if type(key) is not str:
            rating_dict[key] = new_ratings[key]
    return rating_dict

In [3]:
def composite_update(team1, team2, ranks):
    team1_rating = 0
    team1_rd = 0
    team2_rating = 0
    team2_rd = 0
    for key in team1:
        team1_rating += (team1[key].getRating() / len(team1))
        team1_rd += (team1[key].getRd() / len(team1))
    for key in team2:
        team2_rating += (team2[key].getRating() / len(team2))
        team2_rd += (team2[key].getRd() / len(team2))
    for key in team1:
        if type(key) is not str:
            team1[key].update_player([team2_rating], [team2_rd], [ranks[0]])
    for key in team2:
        if type(key) is not str:
            team2[key].update_player([team1_rating], [team1_rd], [ranks[1]])

In [9]:
match_groups = match_outcomes.groupby('match_id')


for e,group in enumerate(match_groups):
    break
print('The group key(match_id):',group[0],'\n')
print('The group data with dire on top row and radiant on bottom.\n',group[1])

The group key(match_id): 1636204962 

The group data with dire on top row and radiant on bottom.
      match_id  account_id_0  account_id_1  account_id_2  account_id_3  \
0  1636204962         34549             0             0     -51743434   
1  1636204962             0         61598        138825             0   

   account_id_4  start_time  parser_version  win  rad  
0    -120875154  1437014585              12    1    0  
1        207232  1437014585              12    0    1  


In [10]:
for e,group in enumerate(match_groups):
    radiant_ids = group[1].iloc[1,1:6] 
    dire_ids = group[1].iloc[0,1:6]
    
    radiant_dict = replace_anonymous(radiant_ids, rating_dict, 'radiant')
    dire_dict = replace_anonymous(dire_ids, rating_dict, 'dire')
    
    if group[1].iloc[1,8] == 1:
        composite_update(radiant_dict, dire_dict, ranks=[1,0]) # 1 - победитель, 0 - проигравший
    else:
        composite_update(radiant_dict, dire_dict, ranks=[0,1])

In [11]:
rating_arr = np.zeros((len(rating_dict.keys()), 3))

for e,i in enumerate(rating_dict.keys()):
    rating_arr[e,0] = i
    rating_arr[e,1] = rating_dict[i].getRating()
    rating_arr[e,2] = rating_dict[i].getRd()

In [12]:
rating_df = pd.DataFrame(rating_arr, columns=['account_id', 'glicko_rating','glicko_rd'])
rating_df

,account_id,glicko_rating,glicko_rd
0,34549.0,1559.984054,98.084123
1,0.0,1500.000000,350.000000
2,-97530201.0,1662.310895,290.318965
3,-123447796.0,1527.776350,66.427887
4,-108454938.0,1659.666624,67.280704
...,...,...,...
834221,-85872761.0,1364.614321,263.418643
834222,-35159264.0,1316.988651,252.532254
834223,156640.0,1669.039206,253.734558
834224,-113657327.0,1624.223786,270.163173


In [13]:
filtered_rating_df = rating_df.loc[rating_df['account_id'] > 0]
filtered_rating_df

,account_id,glicko_rating,glicko_rd
0,34549.0,1559.984054,98.084123
5,320093.0,1575.059093,113.279523
7,179628.0,1593.745910,88.074232
9,103832.0,1465.882473,163.756079
13,31123.0,1636.364519,65.544867
...,...,...,...
834210,36698.0,1329.700052,266.827041
834215,122332.0,1388.337222,267.793250
834216,14572.0,1681.694003,252.176641
834223,156640.0,1669.039206,253.734558


In [ ]:
#filtered_rating_df.to_csv("/data/notebook_files/glicko_ratings.csv")

In [4]:
def g(x):
    return 1/(math.sqrt(1 + (3 * x**2)/(math.pi**2)))

def predict_p1_win(s_1, s_2, r_1, r_2):
    A = g(math.sqrt(r_1**2 + r_2**2)) * (s_2 - s_1)
    return 1/(1 + math.e**(A))

def predict_radiant_win(radiant_dict, dire_dict):
    radiant_rating = 0
    radiant_rd = 0
    dire_rating = 0
    dire_rd = 0
    for key in radiant_dict:
        radiant_rating += (radiant_dict[key].getRating() / len(radiant_dict))
        radiant_rd += (radiant_dict[key].getRd() / len(radiant_dict))
    for key in dire_dict:
        dire_rating += (dire_dict[key].getRating() / len(dire_dict))
        dire_rd += (dire_dict[key].getRd() / len(dire_dict))
    
    return predict_p1_win(radiant_rating, dire_rating, radiant_rd, dire_rd)

In [5]:
filtered_rating_df = pd.read_csv("/data/notebook_files/glicko_ratings.csv")

In [6]:
rating_dict = dict()
filtered_rating_df = filtered_rating_df.reset_index()  # make sure indexes pair with number of rows
for index, row in filtered_rating_df.iterrows():
    rating_dict[row.account_id] = g2.Player(rating=row.glicko_rating, rd=row.glicko_rd)

rating_dict

{34549.0: <glicko2.glicko2.Player at 0x7f344ae687c0>,
 320093.0: <glicko2.glicko2.Player at 0x7f34740d0220>,
 179628.0: <glicko2.glicko2.Player at 0x7f344ae68e80>,
 103832.0: <glicko2.glicko2.Player at 0x7f34740371c0>,
 31123.0: <glicko2.glicko2.Player at 0x7f344ae68d00>,
 245558.0: <glicko2.glicko2.Player at 0x7f3474037430>,
 111635.0: <glicko2.glicko2.Player at 0x7f344ae68ee0>,
 161203.0: <glicko2.glicko2.Player at 0x7f344ae68970>,
 183021.0: <glicko2.glicko2.Player at 0x7f344ae689d0>,
 205333.0: <glicko2.glicko2.Player at 0x7f344ae68d60>,
 26336.0: <glicko2.glicko2.Player at 0x7f344ae68b80>,
 42419.0: <glicko2.glicko2.Player at 0x7f344ae68eb0>,
 61083.0: <glicko2.glicko2.Player at 0x7f344ae68cd0>,
 116094.0: <glicko2.glicko2.Player at 0x7f344ae68d30>,
 293752.0: <glicko2.glicko2.Player at 0x7f344ae685e0>,
 192607.0: <glicko2.glicko2.Player at 0x7f344ae68670>,
 62837.0: <glicko2.glicko2.Player at 0x7f344ae68a90>,
 1778.0: <glicko2.glicko2.Player at 0x7f344ae68610>,
 2859.0: <glicko2.

In [7]:
players = pd.read_csv('/data/notebook_files/players.csv')
match = pd.read_csv('/data/notebook_files/match.csv')

In [8]:
# join ratings with the players table
match_data = pd.merge(players, filtered_rating_df, how='left', left_on='account_id', right_on='account_id')
match_data = pd.merge(match_data, match, how='left', left_on='match_id', right_on='match_id')
#frequent_players =  players.query('total_matches >= 50').copy()

# some of these variables don't have order, and I don't want to plot them against skill
#to_drop = ['match_id', 'hero_id','player_slot', 
#           'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 
#           'leaver_status', 'unit_order_none', 'stuns','total_matches',
#           'total_wins', 'trueskill_sigma']

#frequent_players.drop(to_drop, axis=1, inplace=True)

#frequent_players.shape

columns = ["match_id", "account_id", "player_slot", "radiant_win", "glicko_rating", "glicko_rd"]

match_data = match_data[columns]
match_data = match_data.sort_values(by="match_id", ascending=True)
match_data.head(100)

,match_id,account_id,player_slot,radiant_win,glicko_rating,glicko_rd
0,0,0,0,True,NaN,NaN
1,0,1,1,True,1614.339151,85.437450
2,0,0,2,True,NaN,NaN
3,0,2,3,True,1608.568537,137.836768
4,0,3,4,True,1366.019868,106.757556
...,...,...,...,...,...,...
93,9,0,3,False,NaN,NaN
92,9,0,2,False,NaN,NaN
91,9,0,1,False,NaN,NaN
90,9,0,0,False,NaN,NaN


In [9]:
match_groups = match_data.groupby('match_id')

# just to see what this looks like
for e,group in enumerate(match_groups):
    break

players = group[1].sort_values(by='player_slot', ascending=True)
radiant_ids = players.iloc[0:5,1]
dire_ids = players.iloc[5:10,1]

radiant_dict = replace_anonymous(radiant_ids, rating_dict, 'radiant')
dire_dict = replace_anonymous(dire_ids, rating_dict, 'dire')

radiant_rating = 0
radiant_rd = 0
dire_rating = 0
dire_rd = 0
for key in radiant_dict:
    radiant_rating += (radiant_dict[key].getRating() / len(radiant_dict))
    radiant_rd += (radiant_dict[key].getRd() / len(radiant_dict))
for key in dire_dict:
    dire_rating += (dire_dict[key].getRating() / len(dire_dict))
    dire_rd += (dire_dict[key].getRd() / len(dire_dict))

radiant_win_predict = predict_p1_win(radiant_rating, dire_rating, radiant_rd, dire_rd)

correct_predict = False
efficiency = 0
if (radiant_win_predict >= 0.5 and players.iloc[0,3] == True) or (radiant_win_predict <= 0.5 and players.iloc[0,3] == False):
    correct_predict = True
    efficiency = abs(radiant_win_predict - 0.5)
else:
    efficiency = -abs(radiant_win_predict - 0.5)

new_row = pd.DataFrame(data={"match_id": players.iloc[0,0], "radiant_rating": radiant_rating, "radiant_rd": radiant_rd, "dire_rating": dire_rating, "dire_rd": dire_rd, "radiant_win": players.iloc[0,3], "correct_predict": correct_predict, "radiant_win_predict": radiant_win_predict, "efficiency": efficiency}, index=[0])
#results_prediction = pd.concat([results_prediction, new_row], ignore_index=True)
new_row

,match_id,radiant_rating,radiant_rd,dire_rating,dire_rd,radiant_win,correct_predict,radiant_win_predict,efficiency
0,0,1529.642519,110.010591,1669.632035,89.248589,True,False,0.142796,-0.357204


In [10]:
results_prediction = pd.DataFrame(columns=["match_id", "radiant_rating", "radiant_rd", "dire_rating", "dire_rd", "radiant_win", "correct_predict", "radiant_win_predict", "efficiency"])

In [11]:
for e,group in enumerate(match_groups):
    players = group[1].sort_values(by='player_slot', ascending=True)
    radiant_ids = players.iloc[0:5,1]
    dire_ids = players.iloc[5:10,1]

    radiant_dict = replace_anonymous(radiant_ids, rating_dict, 'radiant')
    dire_dict = replace_anonymous(dire_ids, rating_dict, 'dire')

    radiant_rating = 0
    radiant_rd = 0
    dire_rating = 0
    dire_rd = 0
    for key in radiant_dict:
        radiant_rating += (radiant_dict[key].getRating() / len(radiant_dict))
        radiant_rd += (radiant_dict[key].getRd() / len(radiant_dict))
    for key in dire_dict:
        dire_rating += (dire_dict[key].getRating() / len(dire_dict))
        dire_rd += (dire_dict[key].getRd() / len(dire_dict))

    radiant_win_predict = predict_p1_win(radiant_rating, dire_rating, radiant_rd, dire_rd)

    correct_predict = False
    efficiency = 0
    if (radiant_win_predict >= 0.5 and players.iloc[0,3] == True) or (radiant_win_predict <= 0.5 and players.iloc[0,3] == False):
        correct_predict = True
        efficiency = abs(radiant_win_predict - 0.5)
    else:
        efficiency = -abs(radiant_win_predict - 0.5)

    new_row = pd.DataFrame(data={"match_id": players.iloc[0,0], "radiant_rating": radiant_rating, "radiant_rd": radiant_rd, "dire_rating": dire_rating, "dire_rd": dire_rd, "radiant_win": players.iloc[0,3], "correct_predict": correct_predict, "radiant_win_predict": radiant_win_predict, "efficiency": efficiency}, index=[0])
    results_prediction = pd.concat([results_prediction, new_row], ignore_index=True)

    if group[1].iloc[0,3]: # radiant won
        composite_update(radiant_dict, dire_dict, ranks=[1,0]) # for ranks 0 is winner
    else: # dire won
        composite_update(radiant_dict, dire_dict, ranks=[0,1])

In [12]:
results_prediction.head(100)

,match_id,radiant_rating,radiant_rd,dire_rating,dire_rd,radiant_win,correct_predict,radiant_win_predict,efficiency
0,0,1529.642519,110.010591,1669.632035,89.248589,True,False,0.142796,-0.357204
1,1,1512.145580,87.273572,1625.173879,130.108504,False,True,0.212741,0.287259
2,2,1743.431263,267.969467,1500.000000,350.000000,False,False,0.731383,-0.231383
3,3,1388.444815,193.589432,1552.192039,157.638400,False,True,0.233324,0.266676
4,4,1608.739520,109.204700,1588.207625,141.737391,True,True,0.551844,0.051844
...,...,...,...,...,...,...,...,...,...
95,95,1559.492869,68.644689,1600.597001,81.719407,False,True,0.332152,0.167848
96,96,1525.937817,184.281682,1279.888323,247.751560,True,True,0.809278,0.309278
97,97,1344.041095,201.224326,1500.000000,350.000000,True,False,0.331664,-0.168336
98,98,1529.065491,135.299635,1504.571754,203.709297,False,False,0.545292,-0.045292


In [13]:
(results_prediction["efficiency"].sum() / len(results_prediction.index)) * 1000

7.4804253239206036

In [14]:
count = results_prediction.groupby(by="correct_predict", as_index=True)["match_id"].count()
count

In [15]:
# Процент правильных предсказаний
count.iloc[1] / (count.iloc[1] + count.iloc[0])

0.52408

In [16]:
results_prediction.to_csv("/data/notebook_files/glicko_results_prediction_filled.csv")

In [17]:
# transform the updated rating dictionary into a pandas DataFrame
rating_arr = np.zeros((len(rating_dict.keys()), 3))

for e,i in enumerate(rating_dict.keys()):
    rating_arr[e,0] = i
    rating_arr[e,1] = rating_dict[i].getRating()
    rating_arr[e,2] = rating_dict[i].getRd()

In [18]:
rating_df = pd.DataFrame(rating_arr, columns=['account_id', 'glicko_rating','glicko_rd'])
rating_df

,account_id,glicko_rating,glicko_rd
0,34549.0,1484.795617,86.549421
1,320093.0,1575.059093,113.279523
2,179628.0,1593.745910,88.074232
3,103832.0,1557.177153,153.352291
4,31123.0,1620.485009,65.369039
...,...,...,...
251282,36698.0,1561.695821,229.058699
251283,122332.0,1521.947523,239.281897
251284,14572.0,1454.035667,184.011797
251285,156640.0,1504.352380,211.153710


In [19]:
#rating_df.to_csv("/data/notebook_files/glicko_ratings_950k.csv")

In [20]:
filtered_rating_df = pd.read_csv("/data/notebook_files/glicko_ratings.csv")

In [21]:
rating_dict = dict()
filtered_rating_df = filtered_rating_df.reset_index()  # make sure indexes pair with number of rows
for index, row in filtered_rating_df.iterrows():
    rating_dict[row.account_id] = g2.Player(rating=row.glicko_rating, rd=row.glicko_rd)

rating_dict

{34549.0: <glicko2.glicko2.Player at 0x7f344ae68760>,
 320093.0: <glicko2.glicko2.Player at 0x7f344168ae50>,
 179628.0: <glicko2.glicko2.Player at 0x7f344168a7c0>,
 103832.0: <glicko2.glicko2.Player at 0x7f34740d0160>,
 31123.0: <glicko2.glicko2.Player at 0x7f34740d0130>,
 245558.0: <glicko2.glicko2.Player at 0x7f343f9a6e20>,
 111635.0: <glicko2.glicko2.Player at 0x7f344168a850>,
 161203.0: <glicko2.glicko2.Player at 0x7f343f9a6730>,
 183021.0: <glicko2.glicko2.Player at 0x7f34416b73a0>,
 205333.0: <glicko2.glicko2.Player at 0x7f34416b71f0>,
 26336.0: <glicko2.glicko2.Player at 0x7f34416b7220>,
 42419.0: <glicko2.glicko2.Player at 0x7f343ea4d280>,
 61083.0: <glicko2.glicko2.Player at 0x7f343ea4d3a0>,
 116094.0: <glicko2.glicko2.Player at 0x7f34416b7250>,
 293752.0: <glicko2.glicko2.Player at 0x7f34416b7370>,
 192607.0: <glicko2.glicko2.Player at 0x7f34416b72e0>,
 62837.0: <glicko2.glicko2.Player at 0x7f343ea5d4f0>,
 1778.0: <glicko2.glicko2.Player at 0x7f343ea5d910>,
 2859.0: <glicko2.

In [22]:
players = pd.read_csv('/data/notebook_files/players.csv')
match = pd.read_csv('/data/notebook_files/match.csv')

In [23]:
# join ratings with the players table
match_data = pd.merge(players, filtered_rating_df, how='left', left_on='account_id', right_on='account_id')
match_data = pd.merge(match_data, match, how='left', left_on='match_id', right_on='match_id')
#frequent_players =  players.query('total_matches >= 50').copy()

# some of these variables don't have order, and I don't want to plot them against skill
#to_drop = ['match_id', 'hero_id','player_slot', 
#           'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 
#           'leaver_status', 'unit_order_none', 'stuns','total_matches',
#           'total_wins', 'trueskill_sigma']

#frequent_players.drop(to_drop, axis=1, inplace=True)

#frequent_players.shape

columns = ["match_id", "account_id", "player_slot", "radiant_win", "glicko_rating", "glicko_rd"]

match_data = match_data[columns]
match_data.dropna(inplace=True)
match_data = match_data.sort_values(by="match_id", ascending=True)
match_data.head(100)

,match_id,account_id,player_slot,radiant_win,glicko_rating,glicko_rd
1,0,1,1,True,1614.339151,85.437450
3,0,2,3,True,1608.568537,137.836768
4,0,3,4,True,1366.019868,106.757556
5,0,4,128,True,1556.815184,93.110028
7,0,5,130,True,1647.545406,66.549053
...,...,...,...,...,...,...
183,18,98,3,False,1511.210356,82.020359
184,18,99,4,False,1660.693158,73.924135
185,18,100,128,False,1663.840357,61.550975
187,18,101,130,False,1572.462009,70.682972


In [24]:
match_groups = match_data.groupby('match_id')

In [25]:
results_prediction = pd.DataFrame(columns=["match_id", "radiant_rating", "radiant_rd", "dire_rating", "dire_rd", "radiant_win", "correct_predict", "radiant_win_predict", "efficiency"])

In [26]:
for e,group in enumerate(match_groups):
    if len(group[1].index) != 10:
        continue
    
    players = group[1].sort_values(by='player_slot', ascending=True)
    radiant_ids = players.iloc[0:5,1]
    dire_ids = players.iloc[5:10,1]

    radiant_dict = replace_anonymous(radiant_ids, rating_dict, 'radiant')
    dire_dict = replace_anonymous(dire_ids, rating_dict, 'dire')

    radiant_rating = 0
    radiant_rd = 0
    dire_rating = 0
    dire_rd = 0
    for key in radiant_dict:
        radiant_rating += (radiant_dict[key].getRating() / len(radiant_dict))
        radiant_rd += (radiant_dict[key].getRd() / len(radiant_dict))
    for key in dire_dict:
        dire_rating += (dire_dict[key].getRating() / len(dire_dict))
        dire_rd += (dire_dict[key].getRd() / len(dire_dict))

    radiant_win_predict = predict_p1_win(radiant_rating, dire_rating, radiant_rd, dire_rd)

    correct_predict = False
    efficiency = 0
    if (radiant_win_predict >= 0.5 and players.iloc[0,3] == True) or (radiant_win_predict <= 0.5 and players.iloc[0,3] == False):
        correct_predict = True
        efficiency = abs(radiant_win_predict - 0.5)
    else:
        efficiency = -abs(radiant_win_predict - 0.5)

    new_row = pd.DataFrame(data={"match_id": players.iloc[0,0], "radiant_rating": radiant_rating, "radiant_rd": radiant_rd, "dire_rating": dire_rating, "dire_rd": dire_rd, "radiant_win": players.iloc[0,3], "correct_predict": correct_predict, "radiant_win_predict": radiant_win_predict, "efficiency": efficiency}, index=[0])
    results_prediction = pd.concat([results_prediction, new_row], ignore_index=True)

    if group[1].iloc[0,3]: # radiant won
        composite_update(radiant_dict, dire_dict, ranks=[1,0]) # for ranks 0 is winner
    else: # dire won
        composite_update(radiant_dict, dire_dict, ranks=[0,1])

In [27]:
results_prediction

,match_id,radiant_rating,radiant_rd,dire_rating,dire_rd,radiant_win,correct_predict,radiant_win_predict,efficiency
0,110,1621.209100,70.599047,1634.146554,71.310479,False,True,0.441812,0.058188
1,114,1466.400537,64.546217,1518.806130,66.431326,True,False,0.263856,-0.236144
2,115,1463.015552,115.560812,1596.758321,150.686967,True,False,0.217996,-0.282004
3,164,1519.257978,142.597423,1552.347236,81.057448,False,True,0.409537,0.090463
4,183,1613.680647,97.564422,1440.996843,133.406844,True,True,0.869322,0.369322
...,...,...,...,...,...,...,...,...,...
2103,49743,1564.922337,64.184511,1485.649147,132.329320,False,False,0.726626,-0.226626
2104,49767,1515.113185,112.997542,1496.454110,117.205300,False,False,0.551780,-0.051780
2105,49769,1519.293136,111.066949,1498.799726,107.232861,True,True,0.559899,0.059899
2106,49808,1627.114416,82.667986,1522.619203,117.553017,True,True,0.788974,0.288974


In [28]:
(results_prediction["efficiency"].sum() / len(results_prediction.index)) * 1000

9.420152717360798

In [29]:
count = results_prediction.groupby(by="correct_predict", as_index=True)["match_id"].count()
count

In [30]:
# Процент правильных предсказаний
count.iloc[1] / (count.iloc[1] + count.iloc[0])

0.5218216318785579

In [31]:
results_prediction.to_csv("/data/notebook_files/glicko_results_prediction_full_only.csv")